In [14]:
import pickle
import mediapipe as mp
import numpy as np
import pandas as pd

In [16]:
model=pickle.load(open('stance_model.pkl','rb'))

In [17]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [18]:
import cv2
import time

In [19]:
landmarks=[]
for val in range(1,33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [21]:
cap = cv2.VideoCapture(0)

stance=None
probab=0

with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks is not None:
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.pose_landmarks.landmark[30].visibility >= 0.7 and results.pose_landmarks.landmark[29].visibility >= 0.7:

                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles
                    .get_default_pose_landmarks_style())
                
                keypoints=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
                data=pd.DataFrame([keypoints],columns=landmarks)
                stance=model['rf'].predict(data)[0]
                if(stance=='narrow'):
                    probab='%.3f'%(model['rf'].predict_proba(data)[0][0])
                if(stance=='neutral'):
                    probab='%.3f'%(model['rf'].predict_proba(data)[0][1])
                if(stance=='wide'):
                    probab='%.3f'%(model['rf'].predict_proba(data)[0][2])

        cv2.rectangle(image, (0,0), (280,50), (245,117,16), -1)

        #stage data
        cv2.putText(image, 'STANCE', (25,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, stance, (10,40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        #probab data
        cv2.putText(image, 'Probability', (160,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, str(probab), (150,40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                
        cv2.imshow('pose', image)

        k = cv2.waitKey(1) & 0xFF
        if(k == 27):
            break
cap.release()
cv2.destroyAllWindows()